# 🦉COSADAMA Introduction to Data Science Study
일자: 2020-07-28   
작성자: 김가윤   
참고자료: 파이썬으로 데이터 주무르기(민형기), 점프 투 파이썬(박응용), 파이썬 입문과 크롤링 기초 부트캠프(잔재미코딩), COSADAMA Spring Rush Introduction to Data Science(박하람)   
교재: 139 - 158쪽 (깃헙 교안으로 공부하면서, 궁금한 부분은 교재를 참고하세요. 혹은 깃헙 교안을 다 보고 교재를 복습 겸 읽어봐도 됩니다!)   
학습키워드: 웹크롤링, Folium, tqdm

## [프로젝트] 시카고 샌드위치 맛집 분석

* 학습목표
    1. 지난 시간에 배운 HTML/CSS 지식, 정규 표현식, BeautifulSoup과 urllib 등을 활용해 시카고 샌드위치 맛집 분석을 해봅니다.
    2. 샌드위치 맛집의 이름, 샌드위치 이름, 가게의 주소 등의 정보를 얻고, 주소로 Google Maps를 이용해 위도와 경도를 찾아주는 Folium으로 시각화 해봅니다. 
    3. 더불어 웹 크롤링을 하여 정보를 가져올 때에는 꽤 시간이 걸리기 때문에 걸리는 시간을 시각화 해 알려주는 tqdm 모듈도 사용해봅니다.
    
그럼 시작해볼까요~? 🥪

## 1. urllib과 BeautifulSoup으로 사이트에 접근하기   
urllib과 BeautifulSoup으로 시카고 샌드위치 맛집 랭킹을 알려주는 사이트를 불러와 볼게요. 여기에서는 가게이름, 가게 메인 메뉴, 각 가게 소개 페이지를 가져와 볼거에요. 우선 필요한 모듈들을 모두 import 해봅시다.

In [1]:
from urllib.request import urlopen 
from bs4 import BeautifulSoup

* [시카고 샌드위치 맛집 사이트](https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/)

교재에서는 url을 두 부분으로 나누어 놓았어요. 지금은 url이 너무 길어서 나누어 놓은 것이지만, 이렇게 쪼개는 방식은 후에 상대경로를 절대경로로 바꾸기 위해서 알아두어야 할 필요가 있습니다.
* [절대경로 & 상대경로](http://jaguarkim2108.blogspot.com/2014/04/html_9450.html) 개념만 여기서 살짝 이해하고 다음 순서에 본격적으로 알아볼게요.

이후에 html을 읽어와 보기로 해요.

In [2]:
url_base = 'https://www.chicagomag.com'
url_sub = '/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
url = url_base + url_sub
html = urlopen(url)

soup = BeautifulSoup(html, 'html.parser')
# soup

그 다음 크롬의 개발자 도구를 확인해 우선 1위의 BLT와 다른 정보들이 담겨있는 태그와 클라스가 무엇인지 확인해봅니다. div 태그에 class는 sammy네요. find_all을 사용해 가져와볼게요.

In [3]:
print(soup.find_all('div', class_='sammy'))

[<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br>
Old Oak Tap<br>
<em>Read more</em> </br></br></a></div>
</div>, <div class="sammy" style="position: relative;">
<div class="sammyRank">2</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/"><b>Fried Bologna</b><br/>
Au Cheval<br/>
<em>Read more</em> </a></div>
</div>, <div class="sammy" style="position: relative;">
<div class="sammyRank">3</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/"><b>Woodland Mushroom</b><br/>
Xoco<br/>
<em>Read more</em> </a></div>
</div>, <div class="sammy" style="position: relative;">
<div class="sammyRank">4</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Al

잘 불러와졌네요! 1) 샌드위치 이름 2) 샌드위치 가게 이름 3) 사이트 주소 4) 랭킹까지 모두 들어있죠? 그런데 너무 다양하게 많이 적혀져 있으니, 이름만 우선 get_text()로 모두 가져와 볼게요. for문으로 이를 출력해보겠습니다. len()을 사용해 정확히 50개가 맞는지도 살펴보구요.

In [4]:
len(soup.find_all('div', class_='sammy'))

50

In [5]:
tmp = soup.find_all('div', class_='sammy')[0] # temporary
tmp

<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br>
Old Oak Tap<br>
<em>Read more</em> </br></br></a></div>
</div>

리스트 형태에서 가장 첫번째 것을 가져왔더니 저희가 보려고 하는 내용들이 다 들어가 있네요. 이제 내용만 쏙쏙 뽑아볼게요.

## 2. 접근한 웹 페이지에서 원하는 데이터 추출하고 정리하기

In [6]:
tmp = soup.find_all('div', class_='sammy')[0] # temporary, 아까 출력해본 리스트의 첫번째 내용
type(tmp) # type을 알려줘

bs4.element.Tag

find_all로 찾은 결과는 위처럼 bs4.element.Tag 형태의 경우 그 변수에 다시 태그로 찾는 (find, find_all) 명령을 사용할 수 있어요.   
리스트 전체에 적용할 경우(tmp = soup.find_all('div', class_'sammy'))엔 bs4.element.Resultset로 나와서 find를 적용시킬 수 없으니 조심!

In [7]:
tmp.find(class_='sammyRank')

<div class="sammyRank">1</div>

여기서 내용인 Rank만 가져오려면 어떻게 해야할까요? 지난 시간에 배웠죠!   
바로 `get_text()`~! 바로 적용해볼게요.

In [8]:
tmp.find(class_='sammyRank').get_text()

'1'

짠! 등수인 1만 가져왔죠? 가게 이름에도 적용해볼게요.

In [9]:
tmp.find(class_='sammyListing').get_text()

'BLT\r\nOld Oak Tap\nRead more '

\r\n, \n...이것을 제거해주려면 어떤 모듈을 불러야 할까요? 네 맞아요 re~ 🥰
정규표현식을 사용해서 없애줘야 겠죠?

In [10]:
import re

tmp_string = tmp.find(class_='sammyListing').get_text()

re.split('\r\n|\n', tmp_string) # |의 의미가 무엇일까요?

['BLT', 'Old Oak Tap', 'Read more ']

In [11]:
# 샌드위치 이름
re.split('\r\n|\n', tmp_string)[0]

'BLT'

In [12]:
# 샌드위치 가게 이름
re.split('\r\n|\n', tmp_string)[1]

'Old Oak Tap'

split을 이용해 메뉴 이름과 가게 이름을 분리했어요.   

#### 절대경로와 상대경로

또 a 태그에서 href 정보를 가지고 클릭하면 연결될 주소도 저장할 수 있어요.

In [13]:
tmp.find('a')['href']

'/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

전체 주소를 출력해서 확인해볼게요.

In [14]:
add = soup.find_all('div', class_='sammy')
for a in add[:15]:
    print(a.find('a')['href'])

/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Als-Deli-Roast-Beef/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Publican-Quality-Meats-PB-L/
http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Hendrickx-Belgian-Bread-Crafter-Belgian-Chicken-Curry-Salad/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Acadia-Lobster-Roll/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Birchwood-Kitchen-Smoked-Salmon-Salad/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Cemitas-Puebla-Atomica-Cemitas/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Nana-Grilled-Laughing-Bird-Shrimp-and-Fried-Oyster-Po-Boy/
http://www.chicagomag.com/Chicago-Magazine/November-2012/

흠...결과가 모두 똑같지 않네요. 아까 제가 말했던 절대경로, 상대경로 부분이 드디어 나왔어요! 하이퍼링크(파란 부분)은 전체 주소가 나와있는 절대 경로이고, 일부 주소만 나와있으면 상대 경로에요. 상대 경로를 이용해 바로 사이트를 열려면, 앞에 https://www.chicagomag.com 을 붙여주어야 합니다. 이때 절대 경로로 잡힌 url은 두고, 상대경로로 잡힌 url을 절대 경로로 바꿔줄 때 사용하는 것이 urllib에 있는 **urljoin** 이라는 명령이에요. 

* [urljoin](https://docs.python.org/3/library/urllib.parse.html)

In [15]:
from urllib.parse import urljoin

urljoin('https://www.chicagomag.com', a.find('a')['href'])

'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Longman-and-Eagle-Wild-Boar-Sloppy-Joe/'

이렇게 말이죠 😎 이제 방법을 모두 알았으니 데이터를 모두 가져와볼게요.

In [16]:
rank = []
main_menu = []
cafe_name = []
url_add =[]

list_soup = soup.find_all('div', 'sammy')

for item in list_soup:
    rank.append(item.find(class_='sammyRank').get_text())
    
    tmp_string = item.find(class_='sammyListing').get_text()
    
    main_menu.append(re.split(('\n|\r\n'), tmp_string)[0])
    cafe_name.append(re.split(('\n|\r\n'), tmp_string)[1])
    
    url_add.append(urljoin(url_base, item.find('a')['href']))

결과가 잘 담겼는지 확인해볼까요?

In [17]:
rank[:10]

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [18]:
main_menu[:10]

['BLT',
 'Fried Bologna',
 'Woodland Mushroom',
 'Roast Beef',
 'PB&L',
 'Belgian Chicken Curry Salad',
 'Lobster Roll',
 'Smoked Salmon Salad',
 'Atomica Cemitas',
 'Grilled Laughing Bird Shrimp and Fried Po’ Boy']

In [19]:
cafe_name[:10]

['Old Oak Tap',
 'Au Cheval',
 'Xoco',
 'Al’s Deli',
 'Publican Quality Meats',
 'Hendrickx Belgian Bread Crafter',
 'Acadia',
 'Birchwood Kitchen',
 'Cemitas Puebla',
 'Nana']

In [20]:
url_add[:10]

['https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/',
 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/',
 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/',
 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Als-Deli-Roast-Beef/',
 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Publican-Quality-Meats-PB-L/',
 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Hendrickx-Belgian-Bread-Crafter-Belgian-Chicken-Curry-Salad/',
 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Acadia-Lobster-Roll/',
 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Birchwood-Kitchen-Smoked-Salmon-Salad/',
 'https://www.chicagomag.com/Chicago-Magazine/Novemb

In [21]:
len(rank), len(main_menu), len(cafe_name), len(url_add)

(50, 50, 50, 50)

순위, 메뉴, 식당 이름, URL까지 모두 잘 담겼네요. 갯수도 모두 50개여서 다 담겼어요! 이제 데이터프레임에 옮겨볼게요.

**DataFrame에 담기**

In [22]:
import pandas as pd

data = {'Rank': rank, 'Menu':main_menu, 'Cafe':cafe_name, 'URL':url_add}
df = pd.DataFrame(data)
df.head()

,Rank,Menu,Cafe,URL
0,1,BLT,Old Oak Tap,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Fried Bologna,Au Cheval,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Woodland Mushroom,Xoco,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Roast Beef,Al’s Deli,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,PB&L,Publican Quality Meats,https://www.chicagomag.com/Chicago-Magazine/No...


깔끔하게 잘 담겼네요. 이제 저희가 모은 데이터를 잘 저장해줘야겠죠? csv 파일로 저장해줄게요 😙

In [23]:
df.to_csv('03. best_sandwiches_list_chicago.csv', sep=',', encoding='utf-8')

## 3. 다수의 웹페이지에서 정보 가져오기

앞서 내용은 웹 페이지에서 순위, 메뉴, 식당 이름, URL을 가져오는 것이었어요. 이제는 URL 주소를 타고 들어가 보면 있는 '가격', '주소'까지 가져와 봅시다 !

* [Old Oak Tap BLT](http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/)

이 페이지를 살펴보면, 아래에 '가격'과 '주소'가 적혀져 있어요. 여기에서 각각 가격과 주소를 얻어보도록 할게요. F12를 눌러 크롬 개발자 도구를 사용해 이 정보가 들어있는 태그를 확인해봅시다. 보니 p 태그에 addy class네요.

먼저 우리가 저장한 csv 파일에 추가해주기 위해 불러와볼게요. 필요한 모듈들도 함께 뽑아와 보구요.

In [24]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen 
import pandas as pd

df = pd.read_csv('03. best_sandwiches_list_chicago.csv', encoding='utf-8', index_col=0)
df.head()

,Rank,Menu,Cafe,URL
0,1,BLT,Old Oak Tap,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Fried Bologna,Au Cheval,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Woodland Mushroom,Xoco,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Roast Beef,Al’s Deli,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,PB&L,Publican Quality Meats,https://www.chicagomag.com/Chicago-Magazine/No...


In [25]:
df['URL'][0]

'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

이 url로 열어서, 가격과 위치를 가져와볼게요.

In [26]:
html = urlopen(df['URL'][0])
soup = BeautifulSoup(html, 'html.parser')

In [27]:
soup.find('p', class_='addy')

<p class="addy">
<em>$10. 2109 W. Chicago Ave., 773-772-0406, <a href="http://www.theoldoaktap.com/">theoldoaktap.com</a></em></p>

In [28]:
price_tmp = soup.find('p', class_='addy').get_text()
price_tmp

'\n$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

이것들을 공백을 기준으로 split 해볼게요.

In [29]:
price_tmp.split()

['$10.', '2109', 'W.', 'Chicago', 'Ave.,', '773-772-0406,', 'theoldoaktap.com']

In [30]:
price_tmp.split()[0]

'$10.'

점을 빼주고 싶어요.

In [31]:
price_tmp.split()[0][:-1]

'$10'

짠~ ㅎㅎ 이제 주소를 한번 볼게요. 뒤에서 두개를 제외하고까지가 주소네요. 다른 주소도 확인해보세요~

In [32]:
price_tmp.split()

['$10.', '2109', 'W.', 'Chicago', 'Ave.,', '773-772-0406,', 'theoldoaktap.com']

여기서 주소만 가져오려면? [1:-2]로 범위를 정해주면 되겠죠.

In [33]:
# 주소만 가져오기
price_tmp.split()[1:-2]

['2109', 'W.', 'Chicago', 'Ave.,']

근데 주소가 다 떨어져있어서 이것을 붙여주려고 해요~ 이럴 땐 join 함수를 사용합니다.

* [str.join()](https://wikidocs.net/13#join)

In [34]:
' '.join(price_tmp.split()[1:-2])

'2109 W. Chicago Ave.,'

주소가 완벽하게 나오네요. 이제 전체 가격과 주소를 모아보고 3개만 뽑아서 확인을 해볼게요.

In [35]:
price = []
address =[]

for n in df.index[:3]:
    html = urlopen(df['URL'][n])
    soup_tmp = BeautifulSoup(html, 'html.parser')

    gettings = soup_tmp.find('p', 'addy').get_text()
    price.append(gettings.split()[0][:-1])
    address.append(' '.join(gettings.split()[1:-2]))

In [36]:
price

['$10', '$9', '$9.50']

In [37]:
address

['2109 W. Chicago Ave.,', '800 W. Randolph St.,', '445 N. Clark St.,']

잘 나오네요 😗

## 4. tqdm 모듈을 사용해 50 페이지에 접근하기

tqdm은 웹 크롤링할 때 상태 진행이 어떻게 되는지 알려주는 바형태의 시각화입니다. 앞서 뽑아낼 정보들이 50페이지를 모두 확인해보면서 추출하는 것이기 때문에 이렇게 얼마나 진행되었는지 알려주는 것이 있으면, 더 편하게 정보를 얻을 수 있겠죠.

terminal에서 conda install -c conda-forge tqdm 해주기   
아래와 같이 tqdm을 import 해주고, 앞서 한 대로 50 페이지를 모두 들어가 가격과 주소를 가져와 보도록 할게요. (코드를 그냥 쓰기 전에 먼저 코드를 어떻게 쓸 지 생각해보고 보는 것이 좋습니다)

In [38]:
from tqdm import tqdm_notebook

price = []
address = []

for n in tqdm_notebook(df.index): 
    html = urlopen(df['URL'][n])
    soup_tmp = BeautifulSoup(html, 'html.parser')

    gettings = soup_tmp.find('p', 'addy').get_text()
    price.append(gettings.split()[0][:-1])
    address.append(' '.join(gettings.split()[1:-2]))

C:\Users\Owner\anaconda3\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


제대로 가져와졌는지 확인해볼게요~?

In [39]:
price[:10]

['$10', '$9', '$9.50', '$9.40', '$10', '$7.25', '$16', '$10', '$9', '$17']

In [40]:
address[:30]

['2109 W. Chicago Ave.,',
 '800 W. Randolph St.,',
 '445 N. Clark St.,',
 '914 Noyes St., Evanston,',
 '825 W. Fulton Mkt.,',
 '100 E. Walton',
 '1639 S. Wabash Ave.,',
 '2211 W. North Ave.,',
 '3619 W. North Ave.,',
 '3267 S. Halsted St.,',
 '2537 N. Kedzie Blvd.,',
 'Multiple',
 '3124 N. Broadway,',
 '3455 N. Southport Ave.,',
 '2657 N. Kedzie Ave.,',
 '1120 W. Grand Ave.,',
 '1141 S. Jefferson St.,',
 '333 E. Benton Pl.,',
 '1411 N. Wells St.,',
 '1747 N. Damen Ave.,',
 '3209 W. Irving Park',
 'Multiple',
 '5347 N. Clark St.,',
 '2954 W. Irving Park Rd.,',
 'Multiple',
 '191 Skokie Valley Rd., Highland Park,',
 'Multiple',
 '1818 W. Wilson Ave.,',
 '2517 W. Division St.,',
 '218 W. Kinzie']

어, 그런데 여기 multiple이란게 있죠. 샌드위치 가게가 체인점이라 여러군데에 있나봐요. 일단 이것을 알아두시고 어떻게 처리할 지 고민을 해봅시다.

In [41]:
len(price), len(address)

(50, 50)

다 잘 가져온 것 같네요. 이 정보들을 데이터프레임에 새로운 컬럼에 넣어볼게요.

In [42]:
df['Price'] = price 
df['Address'] = address

df.head()

,Rank,Menu,Cafe,URL,Price,Address
0,1,BLT,Old Oak Tap,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"2109 W. Chicago Ave.,"
1,2,Fried Bologna,Au Cheval,https://www.chicagomag.com/Chicago-Magazine/No...,$9,"800 W. Randolph St.,"
2,3,Woodland Mushroom,Xoco,https://www.chicagomag.com/Chicago-Magazine/No...,$9.50,"445 N. Clark St.,"
3,4,Roast Beef,Al’s Deli,https://www.chicagomag.com/Chicago-Magazine/No...,$9.40,"914 Noyes St., Evanston,"
4,5,PB&L,Publican Quality Meats,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"825 W. Fulton Mkt.,"


가장 왼쪽의 인덱스가 거슬리네요. rank를 index로 넣어줄게요.

In [43]:
df.set_index('Rank', inplace=True)
df.head()

,Menu,Cafe,URL,Price,Address
Rank,,,,,
1,BLT,Old Oak Tap,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"2109 W. Chicago Ave.,"
2,Fried Bologna,Au Cheval,https://www.chicagomag.com/Chicago-Magazine/No...,$9,"800 W. Randolph St.,"
3,Woodland Mushroom,Xoco,https://www.chicagomag.com/Chicago-Magazine/No...,$9.50,"445 N. Clark St.,"
4,Roast Beef,Al’s Deli,https://www.chicagomag.com/Chicago-Magazine/No...,$9.40,"914 Noyes St., Evanston,"
5,PB&L,Publican Quality Meats,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"825 W. Fulton Mkt.,"


In [44]:
df.to_csv('03. best_sandwitches_list_chicago2.csv', sep=',', encoding='utf-8')

저희가 원하던 가격과 주소 정보를 다 넣었으니 안전하게 저장해줍니다.

## 5.Google Maps와 Folium으로 맛집 지도 표기

이제 저희가 모았던 주소를 가지고 google maps, folium을 가지고 표시해볼거에요 !! 일단 필요한 모듈들을 다 import 하구요, 저장했던 데이터도 다시 불러와볼게요.

In [45]:
import pandas as pd
import numpy as np
import googlemaps 
import folium
from tqdm import tqdm_notebook

In [46]:
df = pd.read_csv('03. best_sandwitches_list_chicago2.csv', index_col=0) # incex_col 인덱스 지정 : 0의 위치에 있는 걸 index로 할게
df.head()

,Menu,Cafe,URL,Price,Address
Rank,,,,,
1,BLT,Old Oak Tap,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"2109 W. Chicago Ave.,"
2,Fried Bologna,Au Cheval,https://www.chicagomag.com/Chicago-Magazine/No...,$9,"800 W. Randolph St.,"
3,Woodland Mushroom,Xoco,https://www.chicagomag.com/Chicago-Magazine/No...,$9.50,"445 N. Clark St.,"
4,Roast Beef,Al’s Deli,https://www.chicagomag.com/Chicago-Magazine/No...,$9.40,"914 Noyes St., Evanston,"
5,PB&L,Publican Quality Meats,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"825 W. Fulton Mkt.,"


In [47]:
gmaps_key = '*******************************'
gmaps = googlemaps.Client(key=gmaps_key)

더 정확한 정보를 얻기 위해 df의 주소에 시카고라는 주소를 추가해줄거에요.

In [48]:
target_name = df['Address'][1] + ', ' + 'Chicago'
gmaps_output = gmaps.geocode(target_name)
gmaps_output

[{'address_components': [{'long_name': '2109',
    'short_name': '2109',
    'types': ['street_number']},
   {'long_name': 'West Chicago Avenue',
    'short_name': 'W Chicago Ave',
    'types': ['route']},
   {'long_name': 'West Town',
    'short_name': 'West Town',
    'types': ['neighborhood', 'political']},
   {'long_name': 'Chicago',
    'short_name': 'Chicago',
    'types': ['locality', 'political']},
   {'long_name': 'Cook County',
    'short_name': 'Cook County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Illinois',
    'short_name': 'IL',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '60622', 'short_name': '60622', 'types': ['postal_code']},
   {'long_name': '4821',
    'short_name': '4821',
    'types': ['postal_code_suffix']}],
  'formatted_address': '2109 W Chicago Ave, Chicago, IL 60622, USA',
  'geometry': {'loc

여기서 경도와 위도는 geometry라는 키에 있습니다. 원하는 정보를 제대로 가져오기 위해서 한번 봅시다.

In [49]:
gmaps_output[0]  #리스트를 벗긴 딕셔너리 형태 

{'address_components': [{'long_name': '2109',
   'short_name': '2109',
   'types': ['street_number']},
  {'long_name': 'West Chicago Avenue',
   'short_name': 'W Chicago Ave',
   'types': ['route']},
  {'long_name': 'West Town',
   'short_name': 'West Town',
   'types': ['neighborhood', 'political']},
  {'long_name': 'Chicago',
   'short_name': 'Chicago',
   'types': ['locality', 'political']},
  {'long_name': 'Cook County',
   'short_name': 'Cook County',
   'types': ['administrative_area_level_2', 'political']},
  {'long_name': 'Illinois',
   'short_name': 'IL',
   'types': ['administrative_area_level_1', 'political']},
  {'long_name': 'United States',
   'short_name': 'US',
   'types': ['country', 'political']},
  {'long_name': '60622', 'short_name': '60622', 'types': ['postal_code']},
  {'long_name': '4821',
   'short_name': '4821',
   'types': ['postal_code_suffix']}],
 'formatted_address': '2109 W Chicago Ave, Chicago, IL 60622, USA',
 'geometry': {'location': {'lat': 41.8956049,

In [50]:
gmaps_output[0].get('geometry')

{'location': {'lat': 41.8956049, 'lng': -87.67996149999999},
 'location_type': 'ROOFTOP',
 'viewport': {'northeast': {'lat': 41.8969538802915,
   'lng': -87.67861251970848},
  'southwest': {'lat': 41.8942559197085, 'lng': -87.6813104802915}}}

* [dict.get()](https://wikidocs.net/16#key-valueget) : key로 value 얻기

리스트를 벗기면 딕셔너리 형태가 되니, get을 이용해 경도와 위도 정보가 들어있는 곳까지 왔네요. 그럼 경도와 위도 정보는 아래와 같이 찾으면 됩니다.

In [51]:
gmaps_output[0].get('geometry')['location']

{'lat': 41.8956049, 'lng': -87.67996149999999}

In [52]:
gmaps_output[0].get('geometry')['location']['lat']

41.8956049

In [53]:
gmaps_output[0].get('geometry')['location']['lng']

-87.67996149999999

이해가 되시나요? 그렇다면 이제 모든 위치 정보를 뽑아와 볼건데요, 그 전에 중요한 것은 우리가 가져온 주소들 중에 Multiple이라 적혀있던 주소들을 어떻게 처리할까요?

여기에서는 1)for문을 활용해 multiple이 아닌 위도와 경도 정보를 얻기로 하고, 2)정보가 없는 것은 넘파이를 활용해 NaN 표시를 해줄게요. (혹은 우리가 배운 예외처리를 활용하여 처리할 수도 있겠죠 😏...?)

그렇담 코드를 생각해보시고, tqdm 모듈을 활용해 진행상황을 알아보도록 해요 ~ 

In [54]:
lat = []
lng = []

for n in tqdm_notebook(df.index):
    if df['Address'][n] != 'Multiple':
        target_name = df['Address'][n] + ', ' + 'Chicago'
        gmaps_output = gmaps.geocode(target_name)
        location_output = gmaps_output[0].get('geometry')
        lat.append(location_output['location']['lat'])
        lng.append(location_output['location']['lng'])
    else: 
        lat.append(np.nan)
        lng.append(np.nan)

C:\Users\Owner\anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


잘 뽑아졌는지 확인해볼게요 🙄

In [55]:
lat[:15], lng[:15]

([41.8956049,
  41.8846582,
  41.8905226,
  42.0583217,
  41.8866036,
  41.9000869,
  41.8590541,
  41.9102031,
  41.9097558,
  41.8345302,
  41.9276207,
  nan,
  41.9384419,
  41.9451044,
  41.9301107],
 [-87.67996149999999,
  -87.6476668,
  -87.6307834,
  -87.6837479,
  -87.64853649999999,
  -87.6253366,
  -87.6252007,
  -87.68287529999999,
  -87.7176727,
  -87.6456492,
  -87.706792,
  nan,
  -87.64464369999999,
  -87.6636812,
  -87.7072408])

잘 뽑아온 것 같네요, 이것들을 데이터프레임에 칼럼으로 추가해줄거에요.

In [56]:
df['lat'] = lat 
df['lng'] = lng
df[['lat','lng']].head()

,lat,lng
Rank,,
1,41.895605,-87.679961
2,41.884658,-87.647667
3,41.890523,-87.630783
4,42.058322,-87.683748
5,41.886604,-87.648536


그럼 이제 드디어! folium으로 시각화 해볼게요. 우선 가게들의 위도와 경도의 평균으로 지도를 가져와볼거에요. 그래야 우리가 원하는 샌드위치 맛집들이 대략적으로 한 면에 들어오겠죠? 😉

#### follium으로 지도에 표시하기

In [57]:
mapping = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], 
                                      zoom_start=11)
folium.Marker([df['lat'].mean(), df['lng'].mean()], 
                                      popup='center').add_to(mapping)
mapping

marker를 사용해 center를 표시해줬어요. popup은 marker 클릭 시 나오는 이름을 설정해주는 거에요.

그럼 for문을 사용해 주소가 Multiple이 아닐 경우에만 지도에 표기하도록 할게요. popup으로 샌드위치 맛집의 이름도 표기해주고요.

In [58]:
mapping = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], 
                     zoom_start=11)

for n in df.index:
    if df['Address'][n] != 'Multiple':
        folium.Marker([df['lat'][n], df['lng'][n]], 
                                      popup=df['Cafe'][n]).add_to(mapping)

mapping

짠! 완성되었습니다 👏👏 여기까지 달려오신 여러분들을 위해 박수~

## 공부를 마치며 😺

이번 시간에는 크롤링을 통해 데이터를 모으고, 구글 맵으로 표시까지 해보았습니다. 확실히 한 눈에 볼 수 있어 보기 편하네요. 웹 크롤링은 데이터를 모을 때 많이 사용되니 잘 알아두시면 많은 도움이 될거에요. 이번 주차는 내용이 조금 많았죠? 이 모든 내용을 다 외운다! 정복한다! 이런 마음으로 공부하기 보다는 음~ 이런게 있구나~ 이럴땐 이런 걸 사용하는구나~ 하는 느낌으로 공부하시면 마음도 편하고 부담이 덜할거에요. 이번 주도 모두 수고 많으셨습니다. 질문은 항상 두 팔 벌려 환영해요 ~ 그럼 다음 주에 더 재미있는 내용으로 만납시당. 안뇽 ~ 👋